In [3]:
import tensorflow as tf
from keras import layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import itertools

Cargar el conjunto de datos para dividirlo en conjuntos de entrenamiento y prueba

In [4]:
(train_dataset, test_dataset), dataset_info = tfds.load(
    name='caltech101',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
    data_dir='C:\\Users\\ivanc\\PycharmProjects\\scientificProject',
)

Obtener el número de clases en el conjunto de datos

In [5]:
num_classes = dataset_info.features['label'].num_classes

**Definir** una función para preprocesar las imágenes antes de alimentarlas al modelo

In [6]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


Aplicar la función preprocess_image al conjunto de entrenamiento, mezclar los datos y crear lotes de tamaño 32

In [7]:
train_dataset = train_dataset.map(preprocess_image).shuffle(1000).batch(32)

Aplicar la función preprocess_image al conjunto de prueba y crear lotes de tamaño 32

In [8]:
test_dataset = test_dataset.map(preprocess_image).batch(32)


Definir la arquitectura VGG16 utilizando la API secuencial de Keras

In [9]:
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

Compilar el modelo con un optimizador, función de pérdida y métrica de evaluación

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Imprimir el resumen de la arquitectura del modelo

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

Entrenar el modelo en el conjunto de entrenamiento durante 10 épocas y usar el conjunto de prueba para la validación

In [ ]:
model.fit(train_dataset, epochs=1, validation_data=test_dataset)


 1/77 [..............................] - ETA: 1:39:52 - loss: 4.6252 - accuracy: 0.0312

Contar el número de parámetros entrenables en el modelo e imprimirlo

In [ ]:
num_params = model.count_params()
print(f"Número de parámetros entrenables en el modelo: {num_params}")

Evaluar el modelo en el conjunto de prueba e imprimir la precisión

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Precisión en el conjunto de prueba: {accuracy * 100:.2f}%")

Visualize some misclassified images and discuss the possible reasons for misclassifications.

In [ ]:
misclassified_images = []
misclassified_labels = []
for image, label in test_dataset:
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions, axis=1)
    misclassified_idx = np.where(predicted_label != label.numpy())[0]
    for idx in misclassified_idx:
        misclassified_images.append(image[idx])
        misclassified_labels.append(predicted_label[idx])

plt.figure(figsize=(10, 10))
for i in range(len(misclassified_images)):
    plt.subplot(5, 5, i+1)
    plt.imshow(misclassified_images[i])
    plt.title(f"Predicted: {misclassified_labels[i]}")
    plt.axis('off')
plt.show()